In [1]:
from utils import read_jsonl, save_jsonl
import pandas as pd
from pydantic import BaseModel, model_validator, field_validator, Field, ValidationInfo
from typing import List, Dict, Union, Any, Optional
import instructor
from openai import OpenAI
import os
import json

In [2]:
client = instructor.patch(OpenAI(api_key=os.environ['OPENAI_API_KEY']))
MODEL = "gpt-3.5-turbo-0125"

# 🧠 Load data

- reference KBs
- predicted KBs
- Wikidata Properties by Usage Count

In [3]:
pred_kbs = read_jsonl('../../data/prediction.jsonl')
ref_kbs = read_jsonl('../../data/wikidata_entities.jsonl')
popular_properties = pd.read_csv('../../data/wikidata-properties-counts.csv')

In [4]:
print(f"Number of predicted KBs: {len(pred_kbs)}")
print(f"Number of reference KBs: {len(ref_kbs)}")

Number of predicted KBs: 4
Number of reference KBs: 41


## Align our Predicted Entities and Reference Entities

In [5]:
# only take the predictions and references of entities that exist in both
union_entities = set([p['entity_label'] for p in pred_kbs]).intersection(set([r['entity_label'] for r in ref_kbs]))

pred_kbs = [kb for kb in pred_kbs if kb['entity_label'] in union_entities]
pred_kbs = sorted(pred_kbs, key=lambda x: x['entity_label'])

ref_kbs = [kb for kb in ref_kbs if kb['entity_label'] in union_entities]
ref_kbs = sorted(ref_kbs, key=lambda x: x['entity_label'])

print(f"Number of predicted KBs: {len(pred_kbs)}")
print(f"Number of reference KBs: {len(ref_kbs)}")
print("ref: ", [kb['entity_label'] for kb in pred_kbs])
print("pred: ", [kb['entity_label'] for kb in ref_kbs])

Number of predicted KBs: 4
Number of reference KBs: 4
ref:  ['Barack Obama', 'Douglas Adams', 'George Washington', 'Tim Berners-Lee']
pred:  ['Barack Obama', 'Douglas Adams', 'George Washington', 'Tim Berners-Lee']


In [6]:
# ref_kbs[-1]['properties']

In [7]:
ref_kbs[0].keys()

dict_keys(['entity_label', 'properties', 'chunked_content', 'QID'])

In [8]:
pred_kbs[0].keys()

dict_keys(['entity_label', 'properties'])

# 🔎 Filter the Reference KB by the most popular Properties

This is a method to constrain the y label for our model so that the task is easier.

Popular properties are assigned by looking at their usage count across all of Wikidata. [This info is available here](https://www.wikidata.org/wiki/Wikidata:Database_reports/List_of_properties/all)

In [9]:
print(len(popular_properties))
popular_properties.head()

3889


,ID,label,description,Data type[1],Counts[2],largest_number
0,P2860,cites work,citation from one creative or scholarly work t...,WI,"292,583,247 M 390 N",292583247
1,P1545,series ordinal,position of an item in its parent series (most...,S,"175,830,141 Q 2,298 N",175830141
2,P2093,author name string,stores unspecified author or editor name for p...,S,"138,055,438 M 589,477 Q 29,173 R 213 N",138055438
3,P31,instance of,that class of which this subject is a particul...,WI,"114,614,926 M 20 N",114614926
4,P248,stated in,to be used in the references field to refer to...,WI,"98,211,619 R 122 Q 19 N",98211619


In [11]:
print(ref_kbs[0]['entity_label'])
print(f"Num properties: {len(ref_kbs[0]['properties'])}")
list(ref_kbs[0]['properties'].items())[-30:-15] 
# there's a lot of garbage here that isn't even interesting e.g Rotten Tomatoes ID
# but it's mixed in with the good stuff

Barack Obama
Num properties: 348


[('InfluenceWatch influencer', ['person/barack-obama']),
 ('Spotify user ID', ['barackobama']),
 ('KBR person ID', ['14008887']),
 ('CiNii Research ID', ['1140000791660494336']),
 ('Deutsche Synchronkartei person ID', ['4SJPnNOfb']),
 ('ABC News topic ID', ['obama-barack']),
 ('RILM ID', ['30082']),
 ('Great Russian Encyclopedia portal ID', ['obama-barak-1d66f6']),
 ('ScienceDirect topic ID', ['computer-science/president-obama']),
 ('University of Barcelona authority ID', ['981058523280006706']),
 ('Google News topics ID',
  ['CAAqIggKIhxDQkFTRHdvSkwyMHZNREp0YW0xeUVnSmxiaWdBUAE']),
 ('WorldCat Entities ID', ['E39PBJjCKvFkHQ3F9QhY6vw3cP']),
 ('BBC News topic ID', ['cvenzmgywl4t']),
 ('Scopus author ID', ['24587142900']),
 ('Threads username', ['barackobama'])]

In [13]:
def filter_ref_kb_by_top_n_wikiproperties(ref_kbs, wikiproperties, N):

    filtered_ref_kbs = []
        
    for kb in ref_kbs:
        properties = kb['properties']

        top_N = wikiproperties.sort_values(by='largest_number', ascending=False).head(N)

        top_N = top_N['label'].tolist()

        kb['properties'] = {
            key: value for key, value in properties.items() if key in top_N
            }
        filtered_ref_kbs.append(kb)
    
    return filtered_ref_kbs


filtered_ref_kbs = filter_ref_kb_by_top_n_wikiproperties(ref_kbs, popular_properties, 200)
print(f"Num filtered properties: {len(filtered_ref_kbs[0]['properties'])}")
filtered_ref_kbs[0]['properties']


Num filtered properties: 37


{'instance of': ['human'],
 'sex or gender': ['male'],
 'country of citizenship': ['United States of America', 'Kenya'],
 'date of birth': ['+1961-08-04T00:00:00Z'],
 'place of birth': ['Kapiolani Medical Center for Women and Children',
  'Honolulu'],
 'given name': ['Barack', 'Hussein'],
 'mother': ['Stanley Ann Dunham'],
 'father': ['Barack Obama Sr.'],
 'spouse': ['Michelle Obama'],
 'child': ['Sasha Obama', 'Malia Obama'],
 'image': ['President Barack Obama.jpg'],
 'educated at': ['State Elementary School Menteng 01',
  'Punahou School',
  'Occidental College',
  'Columbia University',
  'Harvard Law School',
  'Noelani Elementary School',
  'Centaurus High School',
  'University of Chicago Law School',
  'Harvard University',
  'Nelson High School',
  'King College Prep High School'],
 'position held': ['President of the United States',
  'member of the State Senate of Illinois',
  'United States senator',
  'United States senator',
  'President-elect of the United States'],
 'mem

# 🪬 Define Evaluation Model

In [17]:


class ValidatedProperty(BaseModel):
    property_name: str
    property_value: Union[List[str], str, int, Dict[str, str]]
    property_name_is_valid: bool = Field(
      ...,
        description="A predicted property name is valid if is semantically close " +
                    "to a property name in the reference knowledge base.",
    )
    property_value_is_valid: bool = Field(
      ...,
        description="Whether the property value is generally valid, judged against the " +
                    "reference knowledge base.",
    )
    error_message: Optional[str] = Field(
        None, description="The error message if either property_name and/or property_value is not valid."
    )
    matching_reference_property: Optional[str] = Field(
        ...,
        description="If the predicted property_name is valid, " +
                    "provide the corresponding property_name in the reference knowledge base."
    )


class KnowledgeBase(BaseModel):
    entity_label: str
    properties: Dict[str, Any]


class EvaluationKB(BaseModel):
    predicted_knowledge_base: KnowledgeBase = Field(
        ...,
        description="The predicted knowledge base that must be evaluated against the reference."
    )
    reference_knowledge_base: KnowledgeBase = Field(
        ..., 
        description="The reference knowledge base used for evaluating prediction."
    )
    validated_properties: List[ValidatedProperty] = []


    @model_validator(mode="after")
    def validate_properties(self, context: str) -> "EvaluationKB":

        existing_pred_properties = list(self.predicted_knowledge_base.properties.keys())
        existing_ref_properties = list(self.reference_knowledge_base.properties.keys())

        for predicted_property in self.predicted_knowledge_base.properties:

            # EVALUATE ONE PROPERTY
            resp: ValidatedProperty = client.chat.completions.create(
                response_model=ValidatedProperty,
                messages=[
                    {
                        "role": "user",
                        "content": f"Using your knowledge of the world and the given " +
                        "reference knowledge base, is the following property valid? " +
                        f"\nPredicted Property: {predicted_property}" +
                        f"\n\nReference Knowledge Base: {self.reference_knowledge_base}"
                    }
                ],
                validation_context={
                    "existing_ref_properties": existing_ref_properties,
                    "existing_pred_properties": existing_pred_properties,
                },
                max_retries=3,
                model=MODEL,
            )

            self.validated_properties.append(resp)
        return self

    @field_validator('validated_properties', mode='after')
    @classmethod
    def assert_all_properties_validated(cls, validation_properties: List[ValidatedProperty], info: ValidationInfo):
        existing_pred_properties = info.context.get("existing_pred_properties")
        if len(validation_properties) != len(existing_pred_properties):
            raise ValueError(
                "Number of properties validated does not match number of properties in the prediction knowledge base. " +
                "Number of properties validated: {len(validation_properties)}, " +
                f"Number of properties in the text: {len(existing_pred_properties)}"
                )
        return validation_properties

    @field_validator('validated_properties', mode='after')
    @classmethod
    def assert_matching_ref_in_ref_kb(cls, validation_properties: List[ValidatedProperty], info: ValidationInfo):
        '''
        Make sure that the reference property that is linked to the prediction 
        actually exists in the reference knowledge base.
        '''
        existing_ref_properties = info.context.get("existing_ref_properties")
        for prop in validation_properties:
            if prop.property_name_is_valid:
                if prop.matching_reference_property not in existing_ref_properties:
                    raise ValueError(
                        f"The predicted property name {prop.property_name} was marked valid but the matching_reference_property " +
                        f"{prop.matching_reference_property} does not exist in the reference knowledge base."
                    )
        return validation_properties


    

# Evaluate!

In [18]:
results = []
for idx in range(len(pred_kbs)):

    # 🚨 NOTE: take the text data OUT! It'll fill the prompt otherwise
    chunked_content = filtered_ref_kbs[idx].pop('chunked_content', None) # take the text out 
    QID = filtered_ref_kbs[idx].pop('QID', None)
    eval_info = {
        'predicted_knowledge_base': pred_kbs[idx],
        'reference_knowledge_base': filtered_ref_kbs[idx],
    }
    try:
        result = EvaluationKB(**eval_info)
        results.append(result)
    except Exception as e:
        print(f"Failed to evaluate at KB {idx} with error {e}")

In [19]:
results_json = [r.model_dump() for r in results]
save_jsonl(results_json, '../../data/evaluation_results.jsonl')

Saved to f'../../data/evaluation_results.jsonl


# Look at our Evaluations

In [20]:
results = read_jsonl('../../data/evaluation_results.jsonl')
len(results)

4

In [81]:
def visually_verify_results(result: Dict):
    '''Look at the validation KB, prediction KB, and reference KB side-by-side'''
    print(f"Validating for entity {result['validated_knowledge_base']['entity_label']}")

    for val in result['validated_knowledge_base']['properties']:
        print('\n--------------------------')
        print(f"🤖 Validated Property: {json.dumps(val, indent=4)}")

        matching_pred_property = result['predicted_knowledge_base']['properties'][val['property_name']]
        print(f"What does the prediction KB say? \n{matching_pred_property}")

        # if the predicted property was valid, get the reference the LLM points to
        if val['property_name_is_valid'] is True:  
            matching_ref_property = result['reference_knowledge_base']['properties'][val['matching_reference_property']]
            print(f"✅What does the reference KB say? \n{val['matching_reference_property']}: {matching_ref_property}")
        else:
            print(f"🚨The LLM said this one is not in the reference KB!")

In [82]:
visually_verify_results(results[2])

Validating for entity Tim Berners-Lee

--------------------------
🤖 Validated Property: {
    "property_name": "Full Name",
    "property_value": "Sir Timothy John Berners-Lee",
    "property_name_is_valid": false,
    "property_value_is_valid": true,
    "error_message": null,
    "matching_reference_property": "given name"
}
What does the prediction KB say? 
Sir Timothy John Berners-Lee
🚨The LLM said this one is not in the reference KB!

--------------------------
🤖 Validated Property: {
    "property_name": "Date of Birth",
    "property_value": "8 June 1955",
    "property_name_is_valid": true,
    "property_value_is_valid": true,
    "error_message": null,
    "matching_reference_property": "date of birth"
}
What does the prediction KB say? 
8 June 1955
✅What does the reference KB say? 
date of birth: ['+1955-06-08T00:00:00Z', '+1955-00-00T00:00:00Z']

--------------------------
🤖 Validated Property: {
    "property_name": "Nationality",
    "property_value": "English",
    "prope

In [21]:
def parse_none(s):
    if s is not None:
        return None if s.lower() in ['none', 'null'] else s
    return s

def calc_metrics(result: Dict, metrics: Dict[str, int] = None) -> Dict[str, int]:
    '''Calculate TP, FP, FN for a given result. Add to metrics dict if provided'''
    if metrics is None:
        tp = 0
        fp = 0
        fn = 0
    else:
        tp, fp, fn = metrics['tp'], metrics['fp'], metrics['fn']

    reference_properties_caught = []
    for val in result['validated_knowledge_base']['properties']:

        # true positive
        if val['property_name_is_valid'] is True:  
            tp += 1
            reference_properties_caught.append(val['matching_reference_property'])
        # false positive
        elif val['property_name_is_valid'] is False and (parse_none(val['matching_reference_property']) is None):
            fp += 1

    # false negative -- get all the reference properties that WEREN'T predicted
    for ref_prop_name in result['reference_knowledge_base']['properties'].keys():
        if ref_prop_name not in reference_properties_caught:
            fn += 1

    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1_score = (2 * (precision * recall)) / (precision + recall) if precision + recall > 0 else 0

    print("----------")
    print(f"Validating for entity {result['validated_knowledge_base']['entity_label']}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1_score}")
    print("----------")
    return {'tp': tp, 'fp': fp, 'fn': fn}

metrics = calc_metrics(results[2])

In [ ]:
'''
Initial out for evaluation:

{'predicted_knowledge_base': {'entity_label': 'Alexei Navalny',
  'properties': {'Name': 'Alexei Navalny',
   'Full Name': 'Alexei Anatolyevich Navalny',
   'Date of Birth': '4 June 1976',
   'Date of Death': '16 February 2024',
   'Citizenship': 'Russian',
   'Occupation': 'Politician and Anti-corruption activist',
   'Founded Project': ['RosYama'],
   'Launched Project': ['Anti-Corruption Foundation (FBK)'],
   'Title': 'Investigation and Legal Cases',
   'Description': 'Details about investigations and legal cases involving Alexei Navalny.',
   'Date of death': '16 February 2024',
   'Place of death': 'Yamalo-Nenets in Western Siberia',
   'stated by': 'Alexei Kudrin, Boris Akunin, Mikhail Khodorkovsky, Vladimir Zhirinovsky, Marie Harf, Catherine Ashton, Andreas Schockenhoff, The New York Times, Alexei Venediktov, Levada Center, Leonid Volkov, Alexander Verkhovskiy',
   'criticised by': ['Vladimir Zhirinovsky',
    'United States Department of State Deputy Spokesperson Marie Harf',
    'European Union High Representative Catherine Ashton',
    "Andreas Schockenhoff, Germany's Commissioner for German-Russian Coordination",
    'Levada Center experts',
    'Alexander Verkhovskiy, head of the Moscow-based SOVA hate crimes monitor'],
   'supported by': 'Boris Nemtsov Foundation for Freedom, Geneva Summit for Human Rights and Democracy, Time magazine, Casimir Pulaski Foundation, European Parliament, M100 Media Award, Daniel Roher',
   'Family': ['Married to Yulia Abrosimova',
    'Had two children: daughter Darya (Dasha) Navalnaya and son Zakhar',
    'Daughter began undergraduate studies at Stanford University in September 2019'],
   'Residence': 'Lived primarily in a three-room apartment in Maryino District in southeast Moscow since 1998',
   'Religion': 'Became a member of the Russian Orthodox Church'}},
 'reference_knowledge_base': {'entity_label': 'Alexei Navalny',
  'properties': {'member of': ['Russian Opposition Coordination Council',
    'Yale World Fellows'],
   'sex or gender': ['male'],
   'educated at': ['Finance University under the Government of the Russian Federation',
    'Yale University',
    "Peoples' Friendship University of Russia",
    'Yale World Fellows'],
   'image': ['Alexey Navalny (cropped) 1.jpg'],
   'member of political party': ['Yabloko',
    'Progress Party',
    'Russia of the Future'],
   'Commons category': ['Alexey Navalny'],
   'employer': ['Anti-Corruption Foundation', 'Aeroflot'],
   'date of birth': ['+1976-06-04T00:00:00Z'],
   'religion or worldview': ['Eastern Orthodoxy'],
   'country of citizenship': ['Soviet Union', 'Russia'],
   'field of work': ['politics', 'jurisprudence'],
   'place of birth': ['Butyn'],
   'instance of': ['human'],
   'described by source': ['Lentapedia', 'Navalny'],
   'official website URL': ['https://navalny.com'],
   'given name': ['Alexey'],
   'significant event': ['Yves Rocher case',
    'poisoning of Alexei Navalny',
    'Kirovles trial',
    'incarceration',
    'poisoning of Alexei Navalny',
    'death of Alexei Navalny'],
   'sibling': ['Oleg Navalny'],
   'languages spoken, written or signed': ['Russian', 'English'],
   'name in native language': ["{'text': 'Алексей Анатольевич Навальный', 'language': 'ru'}"],
   'spouse': ['Yulia Navalnaya'],
   'ISNI': ['0000000377416015', '0000000410082812'],
   "topic's main category": ['Category:Alexei Navalny'],
   'on focus list of Wikimedia project': ['WikiProject Human Rights'],
   'family name': ['Navalny'],
   'social media followers': ['+2355803',
    '+2236418',
    '+6460000',
    '+2673187',
    '+2980696',
    '+6370000'],
   'position held': ['Leader of Russia of the Future',
    'Leader of Russia of the Future'],
   'award received': ['Person of the Year',
    'FP Top 100 Global Thinkers',
    'Prize of the Platform of European Memory and Conscience',
    'Time 100',
    'Gold Play Button',
    'Courage Award',
    'Boris Nemtsov Prize',
    'Knight of Freedom Award',
    'Sakharov Prize',
    'M100 Media Award',
    'The BOBs',
    'Silver Play Button',
    'Civil Courage Prize',
    'Günter Walraff award',
    'Bambi Award'],
   'height': ['+1.89'],
   'occupation': ['politician',
    'lawyer',
    'activist',
    'blogger',
    'public figure',
    'activist shareholder',
    'YouTuber',
    'film director',
    'announcer',
    'screenwriter',
    'entrepreneur',
    'political prisoner',
    'jurist',
    'prisoner of conscience'],
   'child': ['Darya Navalnaya', 'Zakhar Navalny'],
   'date of death': ['+2024-02-16T00:00:00Z'],
   'place of death': ['Corrective colony No. 3, YaNAO'],
   'described at URL': ['http://dbpedia.org/resource/Alexei_Navalny'],
   'mother': ['Lyudmila Navalnaya'],
   'father': ['Anatoly Navalny']}},
 'validated_knowledge_base': {'entity_label': 'Alexei Navalny',
  'properties': [{'property_name': 'Name',
    'property_value': 'Alexei Navalny',
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'name in native language'},
   {'property_name': 'Full Name',
    'property_value': 'Alexei Anatolyevich Navalny',
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'given name'},
   {'property_name': 'Date of Birth',
    'property_value': '4 June 1976',
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'date of birth'},
   {'property_name': 'Date of Death',
    'property_value': '16 February 2024',
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'date of death'},
   {'property_name': 'Citizenship',
    'property_value': 'Russian',
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'country of citizenship'},
   {'property_name': 'Occupation',
    'property_value': 'Politician and Anti-corruption activist',
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'occupation'},
   {'property_name': 'Founded Project',
    'property_value': ['RosYama'],
    'is_valid': False,
    'error_message': "The founded project 'RosYama' does not match any reference property.",
    'matching_reference_property': None},
   {'property_name': 'Launched Project',
    'property_value': ['Anti-Corruption Foundation (FBK)'],
    'is_valid': False,
    'error_message': "The launched project 'Anti-Corruption Foundation (FBK)' does not match any reference property.",
    'matching_reference_property': None},
   {'property_name': 'Title',
    'property_value': 'Investigation and Legal Cases',
    'is_valid': False,
    'error_message': "The title 'Investigation and Legal Cases' does not match any reference property.",
    'matching_reference_property': None},
   {'property_name': 'Description',
    'property_value': 'Details about investigations and legal cases involving Alexei Navalny.',
    'is_valid': False,
    'error_message': 'The description does not match any reference property.',
    'matching_reference_property': None},
   {'property_name': 'Date of death',
    'property_value': '16 February 2024',
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'date of death'},
   {'property_name': 'Place of death',
    'property_value': 'Yamalo-Nenets in Western Siberia',
    'is_valid': False,
    'error_message': "The place of death 'Yamalo-Nenets in Western Siberia' does not match the reference property 'Corrective colony No. 3, YaNAO'.",
    'matching_reference_property': None},
   {'property_name': 'stated by',
    'property_value': ['Alexei Kudrin',
     'Boris Akunin',
     'Mikhail Khodorkovsky',
     'Vladimir Zhirinovsky',
     'Marie Harf',
     'Catherine Ashton',
     'Andreas Schockenhoff',
     'The New York Times',
     'Alexei Venediktov',
     'Levada Center',
     'Leonid Volkov',
     'Alexander Verkhovskiy'],
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'described by source'},
   {'property_name': 'criticised by',
    'property_value': ['Vladimir Zhirinovsky',
     'United States Department of State Deputy Spokesperson Marie Harf',
     'European Union High Representative Catherine Ashton',
     "Andreas Schockenhoff, Germany's Commissioner for German-Russian Coordination",
     'Levada Center experts',
     'Alexander Verkhovskiy, head of the Moscow-based SOVA hate crimes monitor'],
    'is_valid': False,
    'error_message': 'The list of critics does not match any reference property.',
    'matching_reference_property': None},
   {'property_name': 'supported by',
    'property_value': 'Boris Nemtsov Foundation for Freedom, Geneva Summit for Human Rights and Democracy, Time magazine, Casimir Pulaski Foundation, European Parliament, M100 Media Award, Daniel Roher',
    'is_valid': False,
    'error_message': 'The list of supporters does not match any reference property.',
    'matching_reference_property': None},
   {'property_name': 'Family',
    'property_value': ['Married to Yulia Abrosimova',
     'Had two children: daughter Darya (Dasha) Navalnaya and son Zakhar',
     'Daughter began undergraduate studies at Stanford University in September 2019'],
    'is_valid': False,
    'error_message': 'The family information does not match any reference property.',
    'matching_reference_property': None},
   {'property_name': 'Residence',
    'property_value': 'Lived primarily in a three-room apartment in Maryino District in southeast Moscow since 1998',
    'is_valid': False,
    'error_message': 'The residence information does not match any reference property.',
    'matching_reference_property': None},
   {'property_name': 'Religion',
    'property_value': 'Became a member of the Russian Orthodox Church',
    'is_valid': False,
    'error_message': 'The religion information does not match any reference property.',
    'matching_reference_property': None}]}}
'''

In [ ]:
'''
Validating for entity Alexei Navalny

--------------------------
🤖 Validated Property: {
    "property_name": "Name",
    "property_value": "Alexei Navalny",
    "is_valid": true,
    "error_message": null,
    "matching_reference_property": "name in native language"
}
What does the prediction KB say? 
Alexei Navalny
✅What does the reference KB say? 
name in native language: ["{'text': 'Алексей Анатольевич Навальный', 'language': 'ru'}"]

--------------------------
🤖 Validated Property: {
    "property_name": "Full Name",
    "property_value": "Alexei Anatolyevich Navalny",
    "is_valid": true,
    "error_message": null,
    "matching_reference_property": "given name"
}
What does the prediction KB say? 
Alexei Anatolyevich Navalny
✅What does the reference KB say? 
given name: ['Alexey']

--------------------------
🤖 Validated Property: {
    "property_name": "Date of Birth",
    "property_value": "4 June 1976",
    "is_valid": true,
    "error_message": null,
    "matching_reference_property": "date of birth"
}
What does the prediction KB say? 
4 June 1976
✅What does the reference KB say? 
date of birth: ['+1976-06-04T00:00:00Z']

--------------------------
🤖 Validated Property: {
    "property_name": "Date of Death",
    "property_value": "16 February 2024",
    "is_valid": true,
    "error_message": null,
    "matching_reference_property": "date of death"
}
What does the prediction KB say? 
16 February 2024
✅What does the reference KB say? 
date of death: ['+2024-02-16T00:00:00Z']

--------------------------
🤖 Validated Property: {
    "property_name": "Citizenship",
    "property_value": "Russian",
    "is_valid": true,
    "error_message": null,
    "matching_reference_property": "country of citizenship"
}
What does the prediction KB say? 
Russian
✅What does the reference KB say? 
country of citizenship: ['Soviet Union', 'Russia']

--------------------------
🤖 Validated Property: {
    "property_name": "Occupation",
    "property_value": "Politician and Anti-corruption activist",
    "is_valid": true,
    "error_message": null,
    "matching_reference_property": "occupation"
}
What does the prediction KB say? 
Politician and Anti-corruption activist
✅What does the reference KB say? 
occupation: ['politician', 'lawyer', 'activist', 'blogger', 'public figure', 'activist shareholder', 'YouTuber', 'film director', 'announcer', 'screenwriter', 'entrepreneur', 'political prisoner', 'jurist', 'prisoner of conscience']

--------------------------
🤖 Validated Property: {
    "property_name": "Founded Project",
    "property_value": [
        "RosYama"
    ],
    "is_valid": false,
    "error_message": "The founded project 'RosYama' does not match any reference property.",
    "matching_reference_property": null
}
What does the prediction KB say? 
['RosYama']
🚨The LLM said this one is not in the reference KB!

--------------------------
🤖 Validated Property: {
    "property_name": "Launched Project",
    "property_value": [
        "Anti-Corruption Foundation (FBK)"
    ],
    "is_valid": false,
    "error_message": "The launched project 'Anti-Corruption Foundation (FBK)' does not match any reference property.",
    "matching_reference_property": null
}
What does the prediction KB say? 
['Anti-Corruption Foundation (FBK)']
🚨The LLM said this one is not in the reference KB!

--------------------------
🤖 Validated Property: {
    "property_name": "Title",
    "property_value": "Investigation and Legal Cases",
    "is_valid": false,
    "error_message": "The title 'Investigation and Legal Cases' does not match any reference property.",
    "matching_reference_property": null
}
What does the prediction KB say? 
Investigation and Legal Cases
🚨The LLM said this one is not in the reference KB!

--------------------------
🤖 Validated Property: {
    "property_name": "Description",
    "property_value": "Details about investigations and legal cases involving Alexei Navalny.",
    "is_valid": false,
    "error_message": "The description does not match any reference property.",
    "matching_reference_property": null
}
What does the prediction KB say? 
Details about investigations and legal cases involving Alexei Navalny.
🚨The LLM said this one is not in the reference KB!

--------------------------
🤖 Validated Property: {
    "property_name": "Date of death",
    "property_value": "16 February 2024",
    "is_valid": true,
    "error_message": null,
    "matching_reference_property": "date of death"
}
What does the prediction KB say? 
16 February 2024
✅What does the reference KB say? 
date of death: ['+2024-02-16T00:00:00Z']

--------------------------
🤖 Validated Property: {
    "property_name": "Place of death",
    "property_value": "Yamalo-Nenets in Western Siberia",
    "is_valid": false,
    "error_message": "The place of death 'Yamalo-Nenets in Western Siberia' does not match the reference property 'Corrective colony No. 3, YaNAO'.",
    "matching_reference_property": null
}
What does the prediction KB say? 
Yamalo-Nenets in Western Siberia
🚨The LLM said this one is not in the reference KB!

--------------------------
🤖 Validated Property: {
    "property_name": "stated by",
    "property_value": [
        "Alexei Kudrin",
        "Boris Akunin",
        "Mikhail Khodorkovsky",
        "Vladimir Zhirinovsky",
        "Marie Harf",
        "Catherine Ashton",
        "Andreas Schockenhoff",
        "The New York Times",
        "Alexei Venediktov",
        "Levada Center",
        "Leonid Volkov",
        "Alexander Verkhovskiy"
    ],
    "is_valid": true,
    "error_message": null,
    "matching_reference_property": "described by source"
}
What does the prediction KB say? 
Alexei Kudrin, Boris Akunin, Mikhail Khodorkovsky, Vladimir Zhirinovsky, Marie Harf, Catherine Ashton, Andreas Schockenhoff, The New York Times, Alexei Venediktov, Levada Center, Leonid Volkov, Alexander Verkhovskiy
✅What does the reference KB say? 
described by source: ['Lentapedia', 'Navalny']

--------------------------
🤖 Validated Property: {
    "property_name": "criticised by",
    "property_value": [
        "Vladimir Zhirinovsky",
        "United States Department of State Deputy Spokesperson Marie Harf",
        "European Union High Representative Catherine Ashton",
        "Andreas Schockenhoff, Germany's Commissioner for German-Russian Coordination",
        "Levada Center experts",
        "Alexander Verkhovskiy, head of the Moscow-based SOVA hate crimes monitor"
    ],
    "is_valid": false,
    "error_message": "The list of critics does not match any reference property.",
    "matching_reference_property": null
}
What does the prediction KB say? 
['Vladimir Zhirinovsky', 'United States Department of State Deputy Spokesperson Marie Harf', 'European Union High Representative Catherine Ashton', "Andreas Schockenhoff, Germany's Commissioner for German-Russian Coordination", 'Levada Center experts', 'Alexander Verkhovskiy, head of the Moscow-based SOVA hate crimes monitor']
🚨The LLM said this one is not in the reference KB!

--------------------------
🤖 Validated Property: {
    "property_name": "supported by",
    "property_value": "Boris Nemtsov Foundation for Freedom, Geneva Summit for Human Rights and Democracy, Time magazine, Casimir Pulaski Foundation, European Parliament, M100 Media Award, Daniel Roher",
    "is_valid": false,
    "error_message": "The list of supporters does not match any reference property.",
    "matching_reference_property": null
}
What does the prediction KB say? 
Boris Nemtsov Foundation for Freedom, Geneva Summit for Human Rights and Democracy, Time magazine, Casimir Pulaski Foundation, European Parliament, M100 Media Award, Daniel Roher
🚨The LLM said this one is not in the reference KB!

--------------------------
🤖 Validated Property: {
    "property_name": "Family",
    "property_value": [
        "Married to Yulia Abrosimova",
        "Had two children: daughter Darya (Dasha) Navalnaya and son Zakhar",
        "Daughter began undergraduate studies at Stanford University in September 2019"
    ],
    "is_valid": false,
    "error_message": "The family information does not match any reference property.",
    "matching_reference_property": null
}
What does the prediction KB say? 
['Married to Yulia Abrosimova', 'Had two children: daughter Darya (Dasha) Navalnaya and son Zakhar', 'Daughter began undergraduate studies at Stanford University in September 2019']
🚨The LLM said this one is not in the reference KB!

--------------------------
🤖 Validated Property: {
    "property_name": "Residence",
    "property_value": "Lived primarily in a three-room apartment in Maryino District in southeast Moscow since 1998",
    "is_valid": false,
    "error_message": "The residence information does not match any reference property.",
    "matching_reference_property": null
}
What does the prediction KB say? 
Lived primarily in a three-room apartment in Maryino District in southeast Moscow since 1998
🚨The LLM said this one is not in the reference KB!

--------------------------
🤖 Validated Property: {
    "property_name": "Religion",
    "property_value": "Became a member of the Russian Orthodox Church",
    "is_valid": false,
    "error_message": "The religion information does not match any reference property.",
    "matching_reference_property": null
}
What does the prediction KB say? 
Became a member of the Russian Orthodox Church
🚨The LLM said this one is not in the reference KB!

'''